In [79]:
import numpy as np
import plotly
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)
py = plotly.offline

In [73]:
from scipy.ndimage import gaussian_filter

In [74]:
domain = np.linspace(0, 10, 50)
f = lambda x: 0.1 * x**3 - x**2 + 2
values = f(domain)
noisy = values + np.random.normal(1, 1, 50)
smooth1 = gaussian_filter(noisy, sigma=1)
smooth2 = gaussian_filter(noisy, sigma=3)
scatter = lambda y: Scatter(x=domain, y=y)
py.iplot({
    "data": [scatter(values), 
             Scatter(x=domain, y=noisy),
             Scatter(x=domain, y=smooth1),
             Scatter(x=domain, y=smooth2)],
    "layout": Layout(title="hello world")
})

imu: we have nine observations, and nine hidden variables
acceleration in g, angular speed in degrees per second, and magnetic flux density (magnetic induction) in gauss
perhaps we can ignore rotation?
are these values related to each other?
a single measurement is a best estimate of the actual value
that is, we have a normal distribution of the measured value, centered at the actual value
but we also assume that the actual value at t has some distribution over the actual value at t - 1

posterior prob: probability of random variable given evidence
likelihood: probability of evidence given random variable
posterior prob (is proportional to) likelihood * prior probability

polytree page 528

smoothing: forward-backward algorithm
online setting: fixed-lag window: smoothen d steps behind current step

if we take measurements sufficiently often, we can expect that difference in acceleration will be small
although in fact it might be large due to swinging the body and trembling

but acceleration is a derivative of speed; we'd expect that the speed changes smoothly
it takes a few tens of a seconds to stop walking, we'd need to measure a magnitude more often to make it smooth

finding the most likely sequence: more than just smoothing for single time slices; need joint distribution for time slices
instead use the viterbi algorithm; but does it work with continuous values?

what we covered so far is less applicable as it deals with discrete values
still, could implement that for the rain and umbrella example
or two halves of the room and presence / absence of signal

start by defining the prob distribution:
if it rained, then it 0.7 rains and 0.3 not rains
if it didn't rain, then it 0.3 rains and 0.7 not rains

tod\yes | no rain | rain
no rain | 0.7     | 0.3
rain    | 0.3     | 0.7

foo         | no rain | rain
no umbrella | 0.8     | 0.1
umbrella    | 0.2     | 0.9

this is a matrix and for any state we have a vector

In [123]:
rain_initial = np.array([0.5, 0.5])
rain_p = np.array([[0.7, 0.3], [0.3, 0.7]])
umbrella_p = np.array([[0.8, 0.1], [0.2, 0.9]])
# umbrella_p = np.array([[0.5, 0.5], [0.5, 0.5]])
o_umbrella = np.array([[0.2, 0], [0, 0.9]])
o_no_umbrella = np.array([[0.8, 0], [0, 0.1]])

def o(evidence):
    return o_umbrella if evidence == 1 else o_no_umbrella

In [58]:
def start_states(n):
    return np.random.randint(0, 2, n)

In [50]:
def state_sequence(start, n):
    seq = np.zeros(n, dtype=np.int8)
    seq[0] = start
    for i in range(n - 1):
        previous = seq[i]
        dist = rain_p[:, previous]
        seq[i + 1] = np.random.choice(2, p=dist)
    return seq

In [67]:
np.array([state_sequence(x, 20) for x in start_states(20)])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 0,

In [70]:
def observation(state):
    p_dist = umbrella_p[:,state]
    return np.random.choice(2, p=p_dist)

In [124]:
n = 40
states = state_sequence(0, n)
observations = np.vectorize(observation, otypes=[np.int8])(states)
(states, observations)

(array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int8),
 array([1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=int8))

In [125]:
states == observations

array([False,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False], dtype=bool)

In [104]:
def normalize(array):
    return array / np.sum(array)

In [105]:
def forward(evidence, n):
    res = np.zeros((2, n))
    
    def recurse(i):
        return rain_initial if i == 0 else res[:, i - 1]
    
    for i in xrange(n):
        raw = rain_p.dot(recurse(i)) * umbrella_p[:,evidence[i]]
        res[:, i] = normalize(raw)
        
    return res

In [106]:
# def backward(evidence, n):
#     res = np.zeros((2, n))
    
#     def recurse(i):
#         return np.array([1, 1]) if i == n - 1 else res[:, i + 1]
    
#     for i in xrange(n, 0, -1):
#         j = i - 1
#         raw = rain_p.dot(recurse(j)) * umbrella_p[:,evidence[j]]
#         res[:, j] = normalize(raw)
        
#     return res

In [127]:
def backward(next, evidence):
    return normalize(rain_p.dot(o(evidence)).dot(next))

now smoothing: compute both, then mult and normalize

In [116]:
def smoothing(evidence, n):
    f = forward(evidence, n)
    res = np.zeros((2, n))
    b = np.array([1, 1])
    for i in reversed(xrange(n)):
        res[:, i] = normalize(f[:, i] * b)
        b = backward(b, evidence[i])
    return res

In [128]:
smooth = smoothing(observations, 20)
smooth

array([[ 0.18534056,  0.77298077,  0.7348584 ,  0.09740262,  0.04550849,
         0.10063168,  0.79810194,  0.91365691,  0.92587252,  0.84689613,
         0.30437531,  0.82839174,  0.90707985,  0.82929295,  0.25409185,
         0.54758792,  0.0739294 ,  0.0311918 ,  0.03052196,  0.049717  ],
       [ 0.81465944,  0.22701923,  0.2651416 ,  0.90259738,  0.95449151,
         0.89936832,  0.20189806,  0.08634309,  0.07412748,  0.15310387,
         0.69562469,  0.17160826,  0.09292015,  0.17070705,  0.74590815,
         0.45241208,  0.9260706 ,  0.9688082 ,  0.96947804,  0.950283  ]])

In [134]:
filtering = np.argmax(forward(observations, n), axis=0)
np.sum(states == filtering)

33

In [133]:
smooth = np.argmax(smoothing(observations, n), axis=0)
np.sum(states == smooth)

33

In [135]:
states == smooth

array([False,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False], dtype=bool)

TODO: maybe impl Viterbi's algorithm

how do we test numerical algorithms?

simple alpha beta or g-h filter: works with position/velocity, but what about acceleration etc?

In [1]:
import csv

In [38]:
def convert(row):
    id = row[0]
    rssi = row[1]
    x = row[2]
    y = row[3]
    return (id, int(rssi), (int(x) + 390) / 3, (int(y) + 960) / 3)
    
data = []

with open('rssi.csv', 'rb') as f:
    reader = csv.reader(f)
    data = map(convert, reader)

points = [(row[2], row[3]) for row in data]

print(points[:20])

[(391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947), (391, 947)]


In [52]:
beacons = [
    (350, 890),
    (355, 680),
    (585, 870),
    (630, 710),
    (540, 550),
    (470, 860),
    (365, 573),
    (410, 350),
    (530, 240),
    (635, 380)
]

beacons = map(transform, beacons)

print(beacons)

[(409, 479), (404, 651), (204, 495), (165, 626), (243, 758), (304, 504), (396, 739), (356, 922), (252, 1012), (161, 897)]


In [83]:
from PIL import Image, ImageDraw, ImageFont
# get an image
base = Image.open('map.png')
print(base.size)
fnt = ImageFont.truetype('/Users/piotr/Library/Fonts/Literation Mono Powerline.ttf', 40)

for b in beacon_id:
    img = base.copy()
    d = ImageDraw.Draw(img)
    
    d.text((10,10), b, font=fnt, fill=(0,0,0,0))
    
    for beacon in beacons:
        (x, y) = beacon
        d.ellipse([x, y, x+5, y+5], outline=(0,255,0,0), fill=(0,255,0,0))

    points = [(row[1], row[2], row[3]) for row in beacon_readings[b]]
    print("{} {}".format(b, len(points)))

    for point in points:
        (rssi, x, y) = point
        if rssi >= -87:
            d.ellipse([x, y, x+5, y+5], outline=(255,0,0,0), fill=(255,0,0,0))

    img.show(title=b)



(892, 1262)
17 489
f8 21
62 19
43 2
3 28
2a 29
b8 499


In [87]:
def convert_c(row):
    return (row[0][-2:], float(row[1]), float(row[2]))

with open('coordinates.csv', 'rb') as f:
    reader = csv.reader(f)
    coors = map(convert_c, reader)
    
coors.sort(key = lambda row: row[2])
coors

[('3d', 55.94444938963575, -3.1869836524128914),
 ('62', 55.94449107087541, -3.186941407620907),
 ('f8', 55.944432116316044, -3.186904862523079),
 ('43', 55.94448393625199, -3.1868280842900276),
 ('2a', 55.94443774892113, -3.1867992505431175),
 ('51', 55.94452261340533, -3.1867526471614838),
 ('d7', 55.94444244275808, -3.1867264956235886),
 ('17', 55.94452336441765, -3.1866540759801865),
 ('b8', 55.94449050761571, -3.1866483762860294),
 ('cd', 55.9444578385393, -3.1866151839494705)]

In [57]:
beacon_id = list(set(row[0] for row in data))
beacon_id

['17', 'f8', '62', '43', '3', '2a', 'b8']

In [81]:
rssi = np.fromiter((row[1] for row in data), dtype=np.int16)
rssi[:10]
trace = go.Histogram(x=rssi)
py.offline.iplot([trace])

In [59]:
beacon_readings = {}
for id in beacon_id:
    beacon_readings[id] = []
    
for row in data:
    beacon = row[0]
    beacon_readings[beacon].append(row)
    
print(beacon_readings['17'][:10])

[('17', -93, 391, 947), ('17', -93, 391, 947), ('17', -90, 391, 947), ('17', -91, 391, 947), ('17', -93, 391, 947), ('17', -92, 391, 947), ('17', -89, 391, 947), ('17', -88, 391, 947), ('17', -90, 391, 947), ('17', -88, 391, 947)]


In [51]:
def sub(t1, t2):
    return (t1[0] - t2[0], t1[1] - t2[1])

def transform(beacon):
    upperleft = (280, 200)
    lowerright = (470, 1045)
    res = sub(beacon, upperleft)
    res = (res[0] / 1.15, res[1] / 1.22)
    res = sub(lowerright, res)
    return (int(res[0]), int(res[1]))

ok now make an image for each point and plot
or learn now

ten beacons: vector of 10
regression: output is 2 numbers: coordinates
but we could be after a prob distribution
then could divide image into blocks of one meter
but first try coordinates

but the position vector is not a linear function of signal strength
instead, signal strength is a log of distance
even with distance, position is solution to an equation with a number of circles
neural network is most general, with non linearities

what about bayesian? prob of location given signal prop to prob of signal given location

for each 

1: 350, 890
2: 355, 680
3: 585, 870
4: 630, 710
5: 540, 550
6: 470, 860
7: 365, 573
8: 410, 350
9: 530, 240
10: 635, 380

ok now plot beacons

now need to map from one to another

get coordinates of the rooms

280, 200 | 665, 200
280, 935 | 665, 930

could get as vectors from (280, 200) -> size (385, 730)

(85, 255) (290, 650) -> size (205, 395)

[(252, 282), (250, 394), (127, 293), (103, 378), (151, 463), (188, 298), (244, 451), (220, 570), (157, 628), (101, 554)]

so scale = 1.88

so subtract (280, 200), then divide by 1.88, then to int, then subtract from lower right coordinate


what's next
which beacon is which

In [47]:
(385.0 / 335, 730.0 / 600)

(1.1492537313432836, 1.2166666666666666)